In [ ]:

import pandas as pd
import numpy as np
import string
from nltk.corpus import stopwords
import spacy
from tqdm import tqdm
import copy
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
#df = pd.read_csv('eng-tam_3_train.csv',sep='\t',names=['category','text']) # separator was not given properly
from google.colab import drive
drive.mount('/content/gdrive')
df = pd.read_csv('/content/gdrive/My Drive/eng-tam_3_train.csv') 
#df = pd.read_csv('eng-tam_3_train.csv')
text=df[['text']]
labels=df[['category']]

Mounted at /content/gdrive


In [ ]:
text

,text
0,@Kadhayalla Nijam thaa gay Punda Apo ne confi...
1,love you too Nanba
2,Semma super 👍👍👍👍
3,Ava ka elom kadavuluka
4,Bro gay persons kalyanam panninaa kozhanda pet...
...,...
3856,Antha akka romba caring
3857,@Dan_Lei Nii Pooi Vidurayaa Unakku Yaarum v...
3858,Frnds neenga antha yedathula a irrunthaalum am...
3859,Looking pretty 🤩


In [ ]:
import re
import nltk
nltk.download('punkt', 'stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.lancaster import LancasterStemmer
lancaster_stemmer = LancasterStemmer()
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package punkt to stopwords...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
def take_data_to_shower(tweet):
    tweet=str(tweet)
    
    noises = ['URL', '@USER', '\'ve', 'n\'t', '\'s', '\'m']

    for noise in noises:
        tweet = tweet.replace(noise, '')

    return re.sub(r'[^a-zA-Z]', ' ', tweet)

In [ ]:
def remove_emoticon(text):
        text=str(text)
    
        emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags 
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
        return emoji_pattern.sub(r'', text)

In [ ]:
 def remove_punc(text):
    text=str(text)
    table = str.maketrans("", "", string.punctuation)
    return text.translate(table)

In [ ]:
def tokenize(tweet):
   tweet=str(tweet)
    
   lower_tweet = tweet.lower()
   return word_tokenize(lower_tweet)

In [ ]:
def remove_stop_words(tokens):
   clean_tokens = []
   stopWords = set(stopwords.words('english'))
   for token in tokens:
       if token not in stopWords:
           if token.replace(' ', '') != '':
               if len(token) > 1:
                   clean_tokens.append(token)
   return clean_tokens

In [ ]:
def stem_and_lem(tokens):
   clean_tokens = []
   for token in tokens:
       token = wordnet_lemmatizer.lemmatize(token)
       token = lancaster_stemmer.stem(token)
       if len(token) > 1:
           clean_tokens.append(token)
   return clean_tokens


In [ ]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [ ]:
##EMBEDDING##
clean_texts = copy.deepcopy(text)
tqdm.pandas(desc="Cleaning Data Phase I...")
clean_texts['text'] = text['text'].progress_apply(take_data_to_shower)

tqdm.pandas(desc="removing emoji...")
clean_texts['text'] = text['text'].progress_apply(remove_emoticon)


tqdm.pandas(desc="removing punctuation...")
clean_texts['text'] = text['text'].progress_apply(remove_punc)



tqdm.pandas(desc="Tokenizing Data...")
clean_texts['tokens'] = clean_texts['text'].progress_apply(tokenize)

tqdm.pandas(desc="Cleaning Data Phase II...")
clean_texts['tokens'] = clean_texts['tokens'].progress_apply(remove_stop_words)

tqdm.pandas(desc="Stemming And Lemmatizing")
clean_texts['tokens'] = clean_texts['tokens'].progress_apply(stem_and_lem)



text_vector = clean_texts['tokens'].tolist()

Stemming And Lemmatizing: 100%|██████████| 3861/3861 [00:02<00:00, 1504.14it/s]


In [ ]:

from sklearn.feature_extraction.text import TfidfVectorizer

def tfid(text_vector):
    vectorizer = TfidfVectorizer()
    untokenized_data =[' '.join(tweet) for tweet in tqdm(text_vector, "Vectorizing...")]
    vectorizer = vectorizer.fit(untokenized_data)
    vectors = vectorizer.transform(untokenized_data).toarray()
    return vectors
  
def get_vectors(vectors, labels, keyword):
    if len(vectors) != len(labels):
        print("Unmatching sizes!")
        return
    result = list()
    for vector, label in zip(vectors, labels):
        if label == keyword:
            result.append(vector)
    return result

In [ ]:
vectors_a = tfid(text_vector) # Numerical Vectors A
labels_a = labels['category'].values.tolist() # Subtask A Labels

Vectorizing...: 100%|██████████| 3861/3861 [00:00<00:00, 406032.69it/s]


In [ ]:
vectors_a

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [ ]:
##CLASSIFING##

from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression

In [ ]:
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import VotingClassifier

In [ ]:
def classify(vectors, labels, type="DT"):


  # Random Splitting With Ratio 3 : 1
  train_vectors, test_vectors, train_labels, test_labels = train_test_split(vectors, labels, random_state=5, test_size=0.2)

  # Initialize Model
  classifier = None
  if(type=="MNB"):
    classifier = MultinomialNB(alpha=0.7)
    classifier.fit(train_vectors, train_labels)
  elif(type=="KNN"):
    classifier = KNeighborsClassifier(n_jobs=4)
    params = {'n_neighbors': [3], 'weights':['uniform', 'distance']}
    classifier = GridSearchCV(classifier, params, cv=3, n_jobs=4)
    classifier.fit(train_vectors, train_labels)
    classifier = classifier.best_estimator_
  elif(type=="SVM"):
    classifier = SVC()
    classifier = GridSearchCV(classifier, {'C':[0.001, 0.01, 0.1, 1, 10]}, cv=3, n_jobs=4)
    classifier.fit(train_vectors, train_labels)
    classifier = classifier.best_estimator_
  elif(type=="DT"):
    classifier = DecisionTreeClassifier()
    #params = {'criterion':['gini','entropy']}
    #classifier = GridSearchCV(classifier, params, cv=3, n_jobs=4)
    classifier.fit(train_vectors, train_labels)
    #classifier = classifier.best_estimator_
  elif(type=="RF"):
    classifier = RandomForestClassifier(max_depth=800, min_samples_split=5)
    params = {'n_estimators': [n for n in range(50,200,50)], 'criterion':['gini','entropy'], }
    classifier = GridSearchCV(classifier, params, cv=3, n_jobs=4)
    classifier.fit(train_vectors, train_labels)
    classifier = classifier.best_estimator_
  elif(type=="LR"):
    classifier = LogisticRegression(multi_class='auto', solver='newton-cg')
    classifier = GridSearchCV(classifier, {"C":np.logspace(-3,3,7), "penalty":["l2"]}, cv=3, n_jobs=4)
    classifier.fit(train_vectors, train_labels)
    classifier = classifier.best_estimator_
  elif(type=="XG"):
    classifier=XGBClassifier()
    classifier.fit(train_vectors, train_labels)
  elif(type=="AD"):
    classifier=AdaBoostClassifier()
    classifier.fit(train_vectors, train_labels)
  elif(type=="VE"):
    estimators = []
    model1 = LogisticRegression()
    estimators.append(('logistic', model1))
    model2 = DecisionTreeClassifier()
    estimators.append(('cart', model2))
    classifier = VotingClassifier(estimators)
    classifier.fit(train_vectors, train_labels)

  else:
    print("Wrong Classifier Type!")
    return

  accuracy = accuracy_score(train_labels, classifier.predict(train_vectors))
  print("Training Accuracy:", accuracy)
  test_predictions = classifier.predict(test_vectors)
  accuracy = accuracy_score(test_labels, test_predictions)
  print("Test Accuracy:", accuracy)
  print("Confusion Matrix:", )
  print(confusion_matrix(test_labels, test_predictions))
  print(classification_report([i for i in test_labels], 
                        [i for i in test_predictions]))

In [ ]:
print("\nBuilding Model SVM...")
classify(vectors_a, labels_a, "SVM") # {MNB, KNN, SVM, DT, RF, LR,XG,AD,VE}


Building Model SVM...


/usr/local/lib/python3.7/dist-packages/joblib/externals/loky/process_executor.py:705: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


Training Accuracy: 1.0
Test Accuracy: 0.8809831824062095
Confusion Matrix:
[[  1  66   0]
 [  0 679   0]
 [  0  26   1]]
                        precision    recall  f1-score   support

            Homophobic       1.00      0.01      0.03        67
Non-anti-LGBT+ content       0.88      1.00      0.94       679
           Transphobic       1.00      0.04      0.07        27

              accuracy                           0.88       773
             macro avg       0.96      0.35      0.35       773
          weighted avg       0.90      0.88      0.83       773



In [ ]:
print("\nBuilding Model MNB...")
classify(vectors_a, labels_a, "MNB")


Building Model MNB...
Training Accuracy: 0.8944300518134715
Test Accuracy: 0.8783958602846055
Confusion Matrix:
[[  0  67   0]
 [  0 679   0]
 [  0  27   0]]
                        precision    recall  f1-score   support

            Homophobic       0.00      0.00      0.00        67
Non-anti-LGBT+ content       0.88      1.00      0.94       679
           Transphobic       0.00      0.00      0.00        27

              accuracy                           0.88       773
             macro avg       0.29      0.33      0.31       773
          weighted avg       0.77      0.88      0.82       773



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
print("\nBuilding Model KNN...")
classify(vectors_a, labels_a, "KNN")



Building Model KNN...
Training Accuracy: 1.0
Test Accuracy: 0.8783958602846055
Confusion Matrix:
[[  0  67   0]
 [  0 679   0]
 [  0  27   0]]
                        precision    recall  f1-score   support

            Homophobic       0.00      0.00      0.00        67
Non-anti-LGBT+ content       0.88      1.00      0.94       679
           Transphobic       0.00      0.00      0.00        27

              accuracy                           0.88       773
             macro avg       0.29      0.33      0.31       773
          weighted avg       0.77      0.88      0.82       773



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
print("\nBuilding Model DT...")
classify(vectors_a, labels_a, "DT")



Building Model DT...
Training Accuracy: 1.0
Test Accuracy: 0.8628719275549805
Confusion Matrix:
[[ 15  48   4]
 [ 26 648   5]
 [  0  23   4]]
                        precision    recall  f1-score   support

            Homophobic       0.37      0.22      0.28        67
Non-anti-LGBT+ content       0.90      0.95      0.93       679
           Transphobic       0.31      0.15      0.20        27

              accuracy                           0.86       773
             macro avg       0.52      0.44      0.47       773
          weighted avg       0.83      0.86      0.85       773



In [ ]:
print("\nBuilding Model RF...")
classify(vectors_a, labels_a, "RF")


Building Model RF...
Training Accuracy: 0.9957901554404145
Test Accuracy: 0.8783958602846055
Confusion Matrix:
[[  0  67   0]
 [  0 679   0]
 [  0  27   0]]
                        precision    recall  f1-score   support

            Homophobic       0.00      0.00      0.00        67
Non-anti-LGBT+ content       0.88      1.00      0.94       679
           Transphobic       0.00      0.00      0.00        27

              accuracy                           0.88       773
             macro avg       0.29      0.33      0.31       773
          weighted avg       0.77      0.88      0.82       773



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
print("\nBuilding Model LR...")
classify(vectors_a, labels_a, "LR")


Building Model LR...
Training Accuracy: 0.9996761658031088
Test Accuracy: 0.8796895213454075
Confusion Matrix:
[[  2  63   2]
 [  2 677   0]
 [  0  26   1]]
                        precision    recall  f1-score   support

            Homophobic       0.50      0.03      0.06        67
Non-anti-LGBT+ content       0.88      1.00      0.94       679
           Transphobic       0.33      0.04      0.07        27

              accuracy                           0.88       773
             macro avg       0.57      0.35      0.35       773
          weighted avg       0.83      0.88      0.83       773



In [ ]:
print("\nBuilding Model XGB...")
classify(vectors_a, labels_a, "XG") 


Building Model XGB...
Training Accuracy: 0.9057642487046632
Test Accuracy: 0.8796895213454075
Confusion Matrix:
[[  3  63   1]
 [  1 677   1]
 [  0  27   0]]
                        precision    recall  f1-score   support

            Homophobic       0.75      0.04      0.08        67
Non-anti-LGBT+ content       0.88      1.00      0.94       679
           Transphobic       0.00      0.00      0.00        27

              accuracy                           0.88       773
             macro avg       0.54      0.35      0.34       773
          weighted avg       0.84      0.88      0.83       773



In [ ]:
print("\nBuilding Model AD...")
classify(vectors_a, labels_a, "AD") 


Building Model AD...
Training Accuracy: 0.9080310880829016
Test Accuracy: 0.8758085381630013
Confusion Matrix:
[[  6  60   1]
 [  8 670   1]
 [  1  25   1]]
                        precision    recall  f1-score   support

            Homophobic       0.40      0.09      0.15        67
Non-anti-LGBT+ content       0.89      0.99      0.93       679
           Transphobic       0.33      0.04      0.07        27

              accuracy                           0.88       773
             macro avg       0.54      0.37      0.38       773
          weighted avg       0.83      0.88      0.84       773



In [ ]:
print("\nBuilding Model VE...")
classify(vectors_a, labels_a, "VE")


Building Model VE...
Training Accuracy: 0.9724740932642487
Test Accuracy: 0.8654592496765847
Confusion Matrix:
[[ 15  52   0]
 [ 25 654   0]
 [  0  27   0]]
                        precision    recall  f1-score   support

            Homophobic       0.38      0.22      0.28        67
Non-anti-LGBT+ content       0.89      0.96      0.93       679
           Transphobic       0.00      0.00      0.00        27

              accuracy                           0.87       773
             macro avg       0.42      0.40      0.40       773
          weighted avg       0.82      0.87      0.84       773



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
